1. finnhub에서 가져온 뉴스 데이터 확인

In [23]:
read_path = "./AAPL_extended_news_2025-06-14.csv"
write_path = "./apple_finbert_finnhub.csv"

In [24]:
import pandas as pd

df = pd.read_csv(read_path)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8769 entries, 0 to 8768
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 8769 non-null   int64  
 1   title              8769 non-null   object 
 2   summary            8603 non-null   object 
 3   link               8769 non-null   object 
 4   publisher          0 non-null      float64
 5   category           8769 non-null   object 
 6   pubDate            8767 non-null   object 
 7   image              4929 non-null   object 
 8   related            8769 non-null   object 
 9   source             8769 non-null   object 
 10  collection_period  8769 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 753.7+ KB


In [26]:
df.head(5)

,id,title,summary,link,publisher,category,pubDate,image,related,source,collection_period
0,135270013,3 No-Brainer Artificial Intelligence (AI) Grow...,Finding value in artificial intelligence (AI) ...,https://finnhub.io/api/news?id=3e8de88fcb69df9...,NaN,company,2025-06-14 09:55:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
1,135258199,Apple Reroutes 97% Of Foxconn iPhone Exports F...,Apple Inc. (NASDAQ:AAPL) has reportedly increa...,https://finnhub.io/api/news?id=e6e09272a3b910a...,NaN,company,2025-06-14 01:30:33,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
2,135270015,Why Superintelligent AI Isn’t Taking Over Anyt...,"Despite claims from top names in AI, researche...",https://finnhub.io/api/news?id=d8a8fdff582d1f1...,NaN,company,2025-06-14 01:00:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
3,135257945,Why Apple (AAPL) Dipped More Than Broader Mark...,Apple (AAPL) reached $196.45 at the closing of...,https://finnhub.io/api/news?id=6964bc22b1f8f8c...,NaN,company,2025-06-13 21:45:02,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14
4,135257921,Apple's $4.4B India Play: The Secret iPhone Sh...,"As Trump targets China, Apple quietly reroutes...",https://finnhub.io/api/news?id=7f50a3430d3f330...,NaN,company,2025-06-13 21:06:46,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,AAPL,Yahoo,2025-06-07_2025-06-14


In [27]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

df = pd.read_csv(read_path)

df = df.fillna("")

df["text"] = df["title"] 

def truncate_and_flag(text, tokenizer, max_len=512):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_len)
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    over_flag = 1 if len(tokenizer.encode(text)) > max_len else 0
    return truncated_text, over_flag

df[["text", "over_512"]] = df["text"].apply(
    lambda x: pd.Series(truncate_and_flag(x, tokenizer, max_len=512))
)

texts = df["text"].tolist()
results = finbert(texts, batch_size=8)

pos_scores, neu_scores, neg_scores = [], [], []
for res in results:
    d = {r["label"].lower(): r["score"] for r in res}
    pos_scores.append(d.get("positive", 0.0))
    neu_scores.append(d.get("neutral", 0.0))
    neg_scores.append(d.get("negative", 0.0))

df["finbert_positive"] = pos_scores
df["finbert_neutral"] = neu_scores
df["finbert_negative"] = neg_scores

df.to_csv(write_path, index=False)
print(f"결과 저장 완료: {write_path}")


Device set to use cuda:0
c:\Users\jhh33\anaconda3\envs\pythonProject1\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


결과 저장 완료: ./apple_finbert_finnhub.csv


Text의 길이가 얼마나 영향을 끼치는지 확인

In [13]:
df = pd.read_csv("./apple_finbert_finnhub.csv")

# 512 토큰 넘은 것만 필터링
df_over = df[df["over_512"] == 1]

# 요약 통계 출력
summary = {
    "총 개수": len(df_over),
    "평균 Positive": df_over["finbert_positive"].mean(),
    "평균 Neutral": df_over["finbert_neutral"].mean(),
    "평균 Negative": df_over["finbert_negative"].mean(),
    "Positive > 0.5": (df_over["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df_over["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df_over["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")


총 개수: 8
평균 Positive: 0.30173408985865535
평균 Neutral: 0.6699639637954533
평균 Negative: 0.028301946480496512
Positive > 0.5: 2
Negative > 0.5: 0
Neutral > 0.7: 5


In [14]:
summary = {
    "총 개수": len(df),
    "평균 Positive": df["finbert_positive"].mean(),
    "평균 Neutral": df["finbert_neutral"].mean(),
    "평균 Negative": df["finbert_negative"].mean(),
    "Positive > 0.5": (df["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")

총 개수: 8590
평균 Positive: 0.27216195756895806
평균 Neutral: 0.5354320729773412
평균 Negative: 0.19240596977245797
Positive > 0.5: 2322
Negative > 0.5: 1639
Neutral > 0.7: 4415
